# SR-Wallfollow-Agenten in einer Gitterwelt

### Ressourcen: 
* SREnvironment-Modul für Hintergrundprozesse der Simulation und für die Visualisierung. 
* Template File mit Tests und Aufgabenstellung

### Aufgabe:  
* Implementiert einen SR-Agenten der Wände verfolgen kann in dem gegebenen Jupyter-Notebook `SR-WallFollow-Template.ipnyb`
* Dazu sollten die Funktionen `sense(self, sensors)` und `action(self, x)` in der Klasse `WallFollowAgent` ergänzt werden. 

Folgendes Verhalten soll erfüllt werden:
* Der SR-Agent soll in den ersten sechs gegebenen Leveln (ohne enge Zwischenräume) eine Wand finden und verfolgen. Der Agent sollte sich in jedem Schritt bewegen, also nicht in zwei aufeinander folgenden Zeitschritten an der gleichen Stelle stehen bleiben. 
	
* Zusätzlich soll der Agent, sobald er einmal in direkten Nachbarschaft eines blockierten Feldes gewesen ist, immer in der direkten Nachbarschaft zu einem blockierten Feld bleiben. Das bedeutet, dass er das erste gefundene Objekt bzw. Außenwand entlang fahren soll, ohne dessen Kontur wieder zu verlassen. 
	
* Angenommen Sie dürften den Agenten nun mit einem Gedächtnis ausstatten, bspw. indem Sie die Sensordaten und Aktion aus dem letzten Zeitschritt zwischenspeichern. Wie wäre es nun möglich Ihre Implementierung so zu ändern, dass der Agent auch in Level 7 die Wandverfolgung beherrscht?  

Zur Vorbereitung beinhalten die nächsten Zellen des Notebooks Packages die installiert und importiert werden müssen. Es sollten alle Zellen nacheinander ausgeführt werden.

In [ ]:
# Execute ONCE if ipympl is not installed yet
%pip install ipympl

In [ ]:
# Packages and Config
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["animation.html"] = "jshtml"

Um euren Agenten zu testen haben wir schon diverse Level vorgegeben und euch ein Python Modul `SREnvironment` mitgeliefert. Ihr könnt einen Blick in das Modul werfen, um zu sehen, wie die Gitterwelt und die Simulation oder die Animation funktionieren. Das ist zur Lösung der Aufgabe aber **nicht notwendig!**

Jedes Level besteht aus einen Field, einer Matrix aus 0 und 1, wobei 1 ein belegtes Feld, und 0 ein nicht belegtes Feld ist. Bei der Initialisierung eines neuen Levels wird eine Id, ein Name, die Größe, der Starpunkt und ein Flag, ob es geschlossen ist gegeben. 

Ein nicht geschlossenes Level hat Seitenränder die überschritten werden können. Das Level _"loopt"_ dann, das heißt, der Agent kommt aus der anderen Seite wieder raus. 

Ihr könnt dieses Verhalten in Level 1 Testen, in dem ihr die Default Action von `self.north` zu `self.west` oder `self.east` ändert.

In [ ]:
from is_sr import Simulation, Renderer, Environment
from is_sr.Levels import Level1, Level2, Level3, Level4, Level5, Level6, Level7, levels

Level0 = Environment(0, "Seperate Rooms", (12,12), ((3,5), (7,5)))
Level0.field[ 0, :] = 1
Level0.field[-1, :] = 1
Level0.field[ :, 0] = 1
Level0.field[ :,-1] = 1
Level0.field[ 6, :] = 1
Level0.field[:3,:3] = 1
Level0.field[-5:,-4:] = 1
Level0.field[5,7] = 1
Level0.field[1:3,7] = 1

In der Klasse `Agent8M` sind Basis Funktionen für einen Agenten mit 8 Sensoren gegeben. Außerdem sind einige Attribute gegeben, die dir bei der Bestimmung der nächsten Aktion helfen können:

```
east  = np.array(( 1, 0), dtype = np.intp)
south = np.array(( 0,-1), dtype = np.intp)
west  = np.array((-1, 0), dtype = np.intp)
north = np.array(( 0, 1), dtype = np.intp)

actions = {"E": east, "S": south, "W": west, "N": north}

sensors = np.array((
              (-1, 1), # NW
              ( 0, 1), # N
              ( 1, 1), # NE
              ( 1, 0), # E
              ( 1,-1), # SE
              ( 0,-1), # S
              (-1,-1), # SW
              (-1, 0)  # W
            ), dtype=np.intp)
```

Diese Variablen definieren die Kommandos "E", "S", "W" und "N", welche über das Dictonary `actions` in  Bewegungsrichtungen übersetzt werden. Die `update` Funktion muss in jedem Schritt eines dieser Kommandos zurück geben. Das `sensors` Array definiert die Position der Sensoren, daher beginnen wir wie in der Vorlesung oben links ( Nordwesten NW) und numerieren die Sensoren im Uhrzeigersinn.

Dein eigener Agent soll von dieser Klasse erben und die Funktionalität so erweitern, dass er in der Lage ist einer Wand in einer Gitterwelt zu folgen.

In [ ]:
from is_sr.Agents import Agent8M

Die folgende Klasse beschreibt den Agenten, der die Aufgabe lösen soll. Wir haben euch schon eine Struktur der Funktion gegeben, die ihr aber natürlich auch erweitern könnt. 

Die Funktion `sense(...)` erhält die 8 Sensoreingaben als Inputs. Diese sind von `0` bis `7` in der gleichen Reihenfolge wie in der Vorlesung nummeriert, wobei `sensor[i] == 0` wenn das Feld *i* frei ist, und `sensor[i] == 1` wenn es mit einer Wand belegt ist. Aus den Sensordaten werden die *j* Merkmale `x[j]` aus der Vorlesung bestimmt und zurückgegeben. Wie groß *j* ist, also wie viele Merkmale bestimmt werden, hängt von eurer implementierung ab.

Die Funktion `action(...)` bekommt die Merkmale aus `sense(...)` und soll eine der vier möglichen Aktionen `self.east`, `self.west`, `self.north` oder `self.sout` zurückgeben. Um euch ein Beispiel zu liefern haben wir die default Rückgabe `return self.north` gegeben. Damit ist die nächste Aktion nach Norden zu gehen. In der Update-Funktion wird die von euch gewählte Aktion ausgeführt.

In [ ]:
class WallFollowAgent(Agent8M):
    """
    Wall-Following Agent according to lecture
    """
    
    def __init__(self, memory=False):
        """
        Constructor
        
        Defers construction to Base Class
        """
        Agent8M.__init__(self, memory)

    @classmethod
    def sense(cls, sensors):
        """
        Generate features from environment using sensor input sensors
        s0 s1 s2
        s7 WF s3
        s6 s5 s4
        """

        """
        STUDENT CODE HERE
        """
        
        return ()

    @classmethod
    def action(cls, x, memory = None):
        """
        Select appropriate rule from rule set based on passed feature vector x

        In some levels memory may be allowed and will be filled with the last executed action.
        """

        """
        STUDENT CODE HERE
        """
        
        return "N"

In der nächsten Zelle wird die Simulation und die damit einhergehende Animation erstellt. Bei der Initialisierung der Simulation legt ihr das Level, den Agenten und die Anzahl der Schritte fest.

In [ ]:
# Level-Auswahl: Einfach euer aktuell gewünschtes Level an die Simulation weitergeben
level = Level6
sim = Simulation(level, WallFollowAgent(level.tight), 100)

sensors, cmds, trajectory = sim.run()
ani=Renderer.RenderSimulation(sim, trajectory)

In den letzten Zellen des Notebooks findet ihr die Testklasse, die auf allen Levels testet, ob ihr die Anforderungen erfüllt habt. 

Nutzt die Outputs um eure Implementierung zu überprüfen. 

In [ ]:
from is_sr.Tests import runTests

runTests(levels, WallFollowAgent)